<a href="https://colab.research.google.com/github/inasiza/Credit-card-fraud-detection/blob/main/Inas_CNN_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#loading the datasets
X_train = pd.read_csv('/content/X_train_update.csv')
Y_train = pd.read_csv('/content/Y_train_CVw08PX.csv')

In [32]:
X_train.head()

,Unnamed: 0,designation,description,productid,imageid
0,0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046
1,1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237
2,2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496
4,4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786


In [33]:
!pip install unidecode
!pip install beautifulsoup4


In [ ]:
# Import necessary libraries
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import string
from bs4 import BeautifulSoup
from unidecode import unidecode

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')


# Custom text preprocessing functions
def preprocess(text):
    # Remove digits
    text = ''.join([char for char in text if not char.isdigit()])

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove diacritics
    text = unidecode(text)

    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove URLs
    text = ' '.join([word for word in text.split() if not (word.startswith('http') or word.startswith('www'))])

    # Lowercase
    text = text.lower()

    # Tokenize
    tokens = word_tokenize(text)

    return tokens




In [34]:
# Apply the custom preprocessing function to the 'designation' column
X_train['designation'] = X_train['designation'].apply(preprocess)


<ipython-input-28-4c89482cf7c0>:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [35]:
X_train.head()

,Unnamed: 0,designation,description,productid,imageid
0,0,"[olivia, personalisiertes, notizbuch, seiten, ...",NaN,3804725264,1263597046
1,1,"[journal, des, arts, le, ndeg, du, lart, et, s...",NaN,436067568,1008141237
2,2,"[grand, stylet, ergonomique, bleu, gamepad, ni...",PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,3,"[peluche, donald, europe, disneyland, marionne...",NaN,50418756,457047496
4,4,"[la, guerre, des, tuques]",Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786


In [36]:
from nltk.corpus import stopwords
stops = set(stopwords.words("french"))

In [40]:
# Install NLTK (if not already installed)
!pip install nltk

# Import necessary libraries
import nltk
from nltk.corpus import stopwords

# Download NLTK resources (if not already downloaded)
nltk.download('stopwords')

# Define stop words
stops = set(stopwords.words('french'))

# Function to remove stop words from a list of words
def remove_stops(row):
    my_list = row['designation']
    meaningful_words = [w for w in my_list if not w in stops]
    return meaningful_words

# Apply the remove_stops function to the 'designation' column
X_train['designation'] = X_train.apply(remove_stops, axis=1)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
X_train.head()

,Unnamed: 0,designation,description,productid,imageid
0,0,"[olivia, personalisiertes, notizbuch, seiten, ...",NaN,3804725264,1263597046
1,1,"[journal, arts, ndeg, lart, marche, salon, dar...",NaN,436067568,1008141237
2,2,"[grand, stylet, ergonomique, bleu, gamepad, ni...",PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,3,"[peluche, donald, europe, disneyland, marionne...",NaN,50418756,457047496
4,4,"[guerre, tuques]",Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786


In [42]:
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras import layers
#from keras.wrappers.scikit_learn import KerasClassifier
#from sklearn.model_selection import RandomizedSearchCV
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization, RNN,Conv2D
from tensorflow.keras.layers import Input,Bidirectional,GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate,LeakyReLU,Flatten,Reshape,MaxPooling2D
from tensorflow.keras import regularizers
from keras.models import save_model
import h5py

In [43]:
embedding_dim = 100
maxlen = 10

sentences = X_train['designation'].values
y = pd.get_dummies(Y_train['prdtypecode']).values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.20, random_state=0)


tokenizer = Tokenizer( filters='0123456789!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)



vocab_size = len(tokenizer.word_index) + 1


X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [124]:
print(y)

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [45]:
import tensorflow as tf
def macro_soft_f1(y, y_hat):

    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0) # on multiplie la proba prédite d'un classe (y_hat) par son label=> Uniquement les proba des vrai positifs seront non nuls
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16) #  calcul du F1 score , 1e-16 pour ne pas diviser par 0
    cost = 1 - soft_f1 # comme on cherche a maximiser F1_score , et qu'il nous faut une fonction coût à minimiser duce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # on fait la moyenne pour tous les labels du batch
    return macro_cost

def macro_f1(y, y_hat, thresh=0.5):
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [46]:
embedding_dim = 100
maxlen = 10

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.SpatialDropout1D(0.2))
model.add(layers.Conv1D(64, 2, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(27, activation='softmax'))


model.compile(loss=macro_soft_f1, optimizer='adam', metrics=['accuracy',macro_f1])
model.summary()

history = model.fit(X_train, y_train,
                   epochs= 20,
                    verbose=1,
                    validation_data=(X_test, y_test),
                    batch_size= 1000)



loss, accuracy, f1 = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy, f1 = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


model.save_weights('model.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 100)           5889200   
                                                                 
 spatial_dropout1d_1 (Spati  (None, 10, 100)           0         
 alDropout1D)                                                    
                                                                 
 conv1d_1 (Conv1D)           (None, 9, 64)             12864     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_1 (Dense)             (None, 27)                1755      
                                                                 
Total params: 5903819 (22.52 MB)
Trainable params: 590

In [47]:
X_TEST = pd.read_csv('/content/X_test_update.csv')

In [49]:
# Apply the custom preprocessing function to the 'designation' column
X_TEST['designation'] = X_TEST['designation'].apply(preprocess)

In [50]:
# Apply the remove_stops function to the 'designation' column
X_TEST['designation'] = X_TEST.apply(remove_stops, axis=1)

In [53]:
X_TEST.head()

,Unnamed: 0,designation,description,productid,imageid
0,84916,"[folkmanis, puppets, marionnette, theatre, min...",NaN,516376098,1019294171
1,84917,"[porte, flamme, gaxix, flamebringer, gaxix, u,...",NaN,133389013,1274228667
2,84918,"[pompe, filtration, speck, badu]",NaN,4128438366,1295960357
3,84919,"[robot, piscine, electrique]",<p>Ce robot de piscine d&#39;un design innovan...,3929899732,1265224052
4,84920,"[hsm, destructeur, securio, coupe, crois, ?, e...",NaN,152993898,940543690


In [54]:
sentences = X_TEST['designation'].values



tokenizer = Tokenizer( filters='0123456789!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(sentences)
X_TEST = tokenizer.texts_to_sequences(sentences)


vocab_size = len(tokenizer.word_index) + 1

X_TEST = pad_sequences(X_TEST, padding='post', maxlen=maxlen)


In [110]:
predictions = model.predict(X_TEST)

432/432 [==============================] - 1s 2ms/step


In [111]:
print(predictions)
print(predictions.shape)

[[3.2281354e-05 1.0852926e-06 1.0583968e-02 ... 7.1686268e-02
  1.6386723e-14 7.8387554e-09]
 [1.2071238e-05 3.0660114e-08 2.1785849e-03 ... 4.9435679e-04
  1.3570613e-12 3.5904645e-07]
 [6.5358017e-09 2.3710240e-07 9.7805401e-04 ... 2.2476317e-05
  5.4498056e-13 2.1216337e-07]
 ...
 [1.7420234e-22 4.9350221e-21 1.4963884e-11 ... 9.5792620e-06
  1.2117797e-30 1.1742456e-16]
 [2.9018850e-06 1.1153508e-09 1.0708297e-05 ... 5.6011317e-04
  1.1277280e-16 1.6757133e-09]
 [6.7074152e-06 1.0485106e-02 6.2305152e-01 ... 3.3589301e-01
  5.6764429e-09 7.3850663e-07]]
(13812, 27)


In [112]:
predictions = np.argmax(predictions, axis=1)
print(predictions)
print(predictions.shape)

[22 23  5 ... 23 23  2]
(13812,)


In [129]:
B = ['2583','1560','1300','2060','2522','1280','2403','2280','1920','1160','1320','10','2705','1140','2582','40','2585','1302','1281','50','2462','2905','60','2220','1301','1940','1180']
# Convert the elements to integers and then sort
sorted_array = np.sort(np.array(B, dtype=int))

print(sorted_array)

[  10   40   50   60 1140 1160 1180 1280 1281 1300 1301 1302 1320 1560
 1920 1940 2060 2220 2280 2403 2462 2522 2582 2583 2585 2705 2905]


In [136]:
# Replace elements in 'predictions' with their corresponding values from 'sorted_array'
predictions = np.array([sorted_array[i] for i in predictions])

print(predictions)

[2582 2583 1160 ... 2583 2583   50]


In [137]:
# Create a DataFrame for y-test predictions
y_test_df = pd.DataFrame({'ptrtypecode': predictions})

# Add the default index to y-test DataFrame
y_test_df['index'] = range(84916, len(y_test_df)+84916)

# Reorder the columns to match the y-train format
y_test_df = y_test_df[['index', 'ptrtypecode']]

# Change the default index to a specific column (replace 'new_index_column' with the desired column name)
new_index_column = 'index'
y_test_df = y_test_df.set_index(new_index_column)
# Remove the name of the index
y_test_df.index.name = None

In [138]:
y_test_df.head()

,ptrtypecode
84916,2582
84917,2583
84918,1160
84919,2583
84920,2462


In [139]:
y_test_df.to_csv('predictions.csv')